## Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import collections
import logging
import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import core.artificial_signal_generators as sig_gen
import core.plotting as plot
import core.signal_processing as sigp
import core.statistics as stats
import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as prnt

In [ ]:
dbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", env.get_system_signature()[0])

prnt.config_notebook()

# Generate signal

In [ ]:
arma00process = sig_gen.ArmaProcess([], [])

In [ ]:
rets = arma00process.generate_sample(
    {"start": "2000-01-01", "periods": 4*252, "freq": "B"},
    scale=5,
    burnin=20,
    seed=42
)

In [ ]:
price = rets.cumsum()

In [ ]:
rets.name += "_rets"
price.name += "_price"

## Price

In [ ]:
plot.plot_cols(price)

In [ ]:
price_decomp = sigp.get_trend_residual_decomp(price, tau=16)

In [ ]:
price_decomp.head(3)

In [ ]:
plot.plot_cols(price_decomp)

### Price wavelet decomposition

In [ ]:
price_smooth, price_detail = sigp.get_swt(price, wavelet="haar")

In [ ]:
plot.plot_cols(price_detail)

In [ ]:
plot.plot_cols(price_smooth)

In [ ]:
plot.plot_correlation_matrix(price_detail, mode="heatmap")

## Returns

In [ ]:
plot.plot_cols(rets)

In [ ]:
stats.apply_normality_test(rets.to_frame())

In [ ]:
plot.plot_autocorrelation(rets)

In [ ]:
plot.plot_spectrum(rets)

### Returns wavelet decomposition

In [ ]:
rets_smooth, rets_detail = sigp.get_swt(rets, "haar")

In [ ]:
plot.plot_cols(rets_detail)

In [ ]:
plot.plot_cols(rets_detail, mode="renormalize")

In [ ]:
stats.apply_normality_test(rets_detail)

In [ ]:
plot.plot_autocorrelation(rets_detail, title_prefix="Wavelet level ")

In [ ]:
plot.plot_spectrum(rets_detail, title_prefix="Wavelet level ")

In [ ]:
plot.plot_correlation_matrix(rets_detail, mode="heatmap")

### Z-scored returns

In [ ]:
zscored_rets = sigp.get_dyadic_zscored(rets, demean=False)

In [ ]:
plot.plot_cols(zscored_rets)

In [ ]:
stats.apply_normality_test(zscored_rets)

In [ ]:
plot.plot_autocorrelation(zscored_rets, title_prefix="tau exp = ")

In [ ]:
plot.plot_spectrum(zscored_rets, title_prefix="tau exp = ")

# EMAs

In [ ]:
impulse = sig_gen.get_impulse(-252, 3 * 252, tick=1)

In [ ]:
impulse.plot()

In [ ]:
for i in range(1, 6):
    sigp.compute_ema(impulse, tau=40, min_periods=20, depth=i).plot()

In [ ]:
for i in range(1, 6):
    sigp.compute_smooth_moving_average(
        impulse, tau=40, min_periods=20, min_depth=1, max_depth=i
    ).plot()

In [ ]:
for i in range(1, 6):
    sigp.compute_smooth_moving_average(
        impulse, tau=40, min_periods=20, min_depth=i, max_depth=5
    ).plot()

In [ ]:
for i in range(1, 6):
    sigp.compute_rolling_norm(
        impulse, tau=40, min_periods=20, min_depth=1, max_depth=i, p_moment=1
    ).plot()

In [ ]:
for i in np.arange(0.5, 4.5, 0.5):
    sigp.compute_rolling_norm(
        impulse, tau=40, min_periods=20, min_depth=1, max_depth=2, p_moment=i
    ).plot()

# Outliers handling

In [ ]:
np.random.seed(100)
n = 100000
data = np.random.normal(loc=0.0, scale=1.0, size=n)
print(data[:5])

srs = pd.Series(data)
srs.plot(kind="hist")

In [ ]:
def _analyze(srs):
    print(np.isnan(srs).sum())
    srs.plot(kind="hist")
    plt.show()
    pprint.pprint(stats)

In [ ]:
mode = "winsorize"
lower_quantile = 0.01
window = 1000
min_periods = 10
stats = collections.OrderedDict()
srs_out = sigp.process_outliers(
    srs, mode, lower_quantile, window=window, min_periods=min_periods, info=stats
)
#
_analyze(srs_out)

In [ ]:
mode = "winsorize"
lower_quantile = 0.01
upper_quantile = 0.90
window = 1000
min_periods = 10
stats = collections.OrderedDict()
srs_out = sigp.process_outliers(
    srs,
    mode,
    lower_quantile,
    upper_quantile=upper_quantile,
    window=window,
    min_periods=min_periods,
    info=stats,
)
#
_analyze(srs_out)

In [ ]:
mode = "set_to_nan"
lower_quantile = 0.01
window = 1000
min_periods = 10
stats = collections.OrderedDict()
srs_out = sigp.process_outliers(
    srs, mode, lower_quantile, window=window, min_periods=min_periods, info=stats
)
#
_analyze(srs_out)

In [ ]:
mode = "set_to_zero"
lower_quantile = 0.10
window = 1000
min_periods = 10
stats = collections.OrderedDict()
srs_out = sigp.process_outliers(
    srs, mode, lower_quantile, window=window, min_periods=min_periods, info=stats
)
#
_analyze(srs_out)